In [48]:
import sys
import os

from functools import partial
import torch.nn as nn
from torch.autograd import grad
import torch
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
from scipy.interpolate import griddata
import time
from itertools import product, combinations
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.gridspec as gridspec
from tqdm.notebook import tqdm


In [49]:
device = torch.device('cuda:0')
#device = torch.device('cpu')

In [50]:
#Loading the data
data = scipy.io.loadmat("cylinder_nektar_wake.mat")

In [51]:
U_star = data['U_star'] # N x 2 x T
P_star = data['p_star'] # N x T
t_star = data['t'] # T x 1
X_star = data['X_star'] # N x 2

N = X_star.shape[0]
T = t_star.shape[0]

# Rearrange Data
XX = np.tile(X_star[:,0], (1,T)) # N x T
YY = np.tile(X_star[:,1], (1,T)) # N x T
TT = np.tile(t_star, (1,N)).T # N x T

UU = U_star[:,0,:] # N x T
VV = U_star[:,1,:] # N x T
PP = P_star # N x T

x = XX.flatten()[:,None] # NT x 1
y = YY.flatten()[:,None] # NT x 1
t = TT.flatten()[:,None] # NT x 1

u = UU.flatten()[:,None] # NT x 1
v = VV.flatten()[:,None] # NT x 1
p = PP.flatten()[:,None] # NT x 1

print(f"Shape of U: {U_star.shape}")
print(f"Shape of P: {P_star.shape}")
print(f"Shape of t: {t_star.shape}")
print(f"Shape of X: {X_star.shape}")
print(f"Shape of XX: {XX.shape}")
print(f"Shape of YY: {YY.shape}")
print(f"Shape of TT: {TT.shape}")
print(f"Shape of UU: {UU.shape}")
print(f"Shape of VV: {VV.shape}")
print(f"Shape of PP: {PP.shape}")
print("After flattening:")
print(f"Shape of U: {x.shape}")
print(f"Shape of U: {y.shape}")
print(f"Shape of U: {t.shape}")
print(f"Shape of U: {u.shape}")
print(f"Shape of U: {v.shape}")
print(f"Shape of U: {p.shape}")

Shape of U: (5000, 2, 200)
Shape of P: (5000, 200)
Shape of t: (200, 1)
Shape of X: (5000, 2)
Shape of XX: (1, 1000000)
Shape of YY: (1, 1000000)
Shape of TT: (5000, 200)
Shape of UU: (5000, 200)
Shape of VV: (5000, 200)
Shape of PP: (5000, 200)
After flattening:
Shape of U: (1000000, 1)
Shape of U: (1000000, 1)
Shape of U: (1000000, 1)
Shape of U: (1000000, 1)
Shape of U: (1000000, 1)
Shape of U: (1000000, 1)


In [52]:
# Sampling the training points
N_train = 5000
idx = np.random.choice(N*T, N_train, replace = False)
x_train = x[idx,:]
y_train = y[idx,:]
t_train = t[idx,:]
#t_train = np.sort(t_train, axis = 0)
u_train = u[idx,:]
v_train = v[idx,:]

x_train = torch.from_numpy(x_train)
x_train = x_train.to(torch.float32)
x_train.requires_grad = True
y_train = torch.from_numpy(y_train)
y_train = y_train.to(torch.float32)
y_train.requires_grad = True
t_train = torch.from_numpy(t_train)
t_train = t_train.to(torch.float32)
t_train.requires_grad = True
u_train = torch.from_numpy(u_train)
u_train = u_train.to(torch.float32)
u_train.requires_grad = True
v_train = torch.from_numpy(v_train)
v_train = v_train.to(torch.float32)
v_train.requires_grad = True

x_train = x_train.to(device)
y_train = y_train.to(device)
t_train = t_train.to(device)
u_train = u_train.to(device)
v_train = v_train.to(device)

In [53]:
x_train.device, y_train.device, t_train.device, u_train.device, v_train.device, x_train.dtype, t_train.dtype

(device(type='cuda', index=0),
 device(type='cuda', index=0),
 device(type='cuda', index=0),
 device(type='cuda', index=0),
 device(type='cuda', index=0),
 torch.float32,
 torch.float32)

In [54]:
class FCNI(nn.Module): # Forward


    def __init__(self, n_input = 1, n_nodes = 5, n_hl = 3, n_output = 1, act = "tanh"):

        super(FCNI, self).__init__()

        if act == "tanh":
            layers = [nn.Linear(n_input, n_nodes), nn.Tanh()]
            for _ in range(n_hl - 1):
                layers.append(nn.Linear(n_nodes, n_nodes))
                layers.append(nn.Tanh())
            layers.append(nn.Linear(n_nodes, n_output))
        elif act == "gelu":
            layers = [nn.Linear(n_input, n_nodes), nn.GELU()]
            for _ in range(n_hl - 1):
                layers.append(nn.Linear(n_nodes, n_nodes))
                layers.append(nn.GELU())
            layers.append(nn.Linear(n_nodes, n_output))
        else:
            layers = [nn.Linear(n_input, n_nodes), nn.ReLU()]
            for _ in range(n_hl - 1):
                layers.append(nn.Linear(n_nodes, n_nodes))
                layers.append(nn.ReLU())
            layers.append(nn.Linear(n_nodes, n_output))

        self.layers = nn.Sequential(*layers)
        self.lambda_1 = nn.Parameter(torch.tensor([0.0], requires_grad = True))
        self.lambda_2 = nn.Parameter(torch.tensor([0.0], requires_grad = True))

    def forward(self, x):
        return self.layers(x)

def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_normal_(m.weight)
        m.bias.data.fill_(0.0)

model_i = FCNI(n_input = 3, n_nodes = 20, n_hl = 8, n_output = 2, act = 'tanh')
model_i.apply(init_weights)
model_i.to(device)

opt = torch.optim.Adam(model_i.parameters(), lr = 1e-3)
iter = 0
model_i

FCNI(
  (layers): Sequential(
    (0): Linear(in_features=3, out_features=20, bias=True)
    (1): Tanh()
    (2): Linear(in_features=20, out_features=20, bias=True)
    (3): Tanh()
    (4): Linear(in_features=20, out_features=20, bias=True)
    (5): Tanh()
    (6): Linear(in_features=20, out_features=20, bias=True)
    (7): Tanh()
    (8): Linear(in_features=20, out_features=20, bias=True)
    (9): Tanh()
    (10): Linear(in_features=20, out_features=20, bias=True)
    (11): Tanh()
    (12): Linear(in_features=20, out_features=20, bias=True)
    (13): Tanh()
    (14): Linear(in_features=20, out_features=20, bias=True)
    (15): Tanh()
    (16): Linear(in_features=20, out_features=2, bias=True)
  )
)

In [55]:
def compute_partials(model, x, y, t):

    # x = torch.squeeze(x)
    # y = torch.squeeze(y)
    # t = torch.squeeze(t)
    # d_train = torch.concat((x.view(-1, 1), y.view(-1, 1), t.view(-1, 1)), axis = 1)
    # d_train = torch.unsqueeze(d_train, 0)
    # pred = model(d_train)
    # pred = torch.squeeze(pred)
    # psi = pred[: , 0]
    # p = pred[:, 1]
    # psi = torch.squeeze(psi)
    # p = torch.squeeze(p)
    # u =  grad(psi, y, torch.ones_like(y), create_graph = True, retain_graph = True)[0]
    # v = -1.*grad(psi, x, torch.ones_like(x), create_graph = True, retain_graph = True)[0]


    # #computing physics for u
    # u_t = grad(u, t, torch.ones_like(u), create_graph = True, retain_graph = True)[0]
    # u_x = grad(u, x, torch.ones_like(u), create_graph = True, retain_graph = True)[0]
    # u_y = grad(u, y, torch.ones_like(u), create_graph = True, retain_graph = True)[0]
    # u_xx = grad(u_x, x, torch.ones_like(u), create_graph = True, retain_graph = True)[0]
    # u_yy = grad(u_y, y, torch.ones_like(u), create_graph = True, retain_graph = True)[0]

    # #computing physics for v
    # v_t = grad(v, t, torch.ones_like(v), create_graph = True, retain_graph = True)[0]
    # v_x = grad(v, x, torch.ones_like(v), create_graph = True, retain_graph = True)[0]
    # v_y = grad(v, y, torch.ones_like(v), create_graph = True, retain_graph = True)[0]
    # v_xx = grad(v_x, x, torch.ones_like(v), create_graph = True, retain_graph = True)[0]
    # v_yy = grad(v_y, y, torch.ones_like(v), create_graph = True, retain_graph = True)[0]

    # #computing physics for p
    # p_x = grad(p, x, torch.ones_like(y), create_graph = True, retain_graph = True)[0]
    # p_y = grad(p, y, torch.ones_like(y), create_graph = True, retain_graph = True)[0]

    # f_u = u_t + model.lambda_1*(u*u_x + v*u_y) + p_x - model.lambda_2*(u_xx + u_yy)
    # f_v = v_t + model.lambda_1*(u*v_x + v*v_y) + p_y - model.lambda_2*(v_xx + v_yy)
    res = model(torch.hstack((x, y, t)))
    psi, p = res[:, 0:1], res[:, 1:2]

    u = torch.autograd.grad(psi, y, grad_outputs=torch.ones_like(psi), create_graph=True)[0] #retain_graph=True,
    v = -1.*torch.autograd.grad(psi, x, grad_outputs=torch.ones_like(psi), create_graph=True)[0]

    u_x = torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u), create_graph=True)[0]
    u_xx = torch.autograd.grad(u_x, x, grad_outputs=torch.ones_like(u_x), create_graph=True)[0]
    u_y = torch.autograd.grad(u, y, grad_outputs=torch.ones_like(u), create_graph=True)[0]
    u_yy = torch.autograd.grad(u_y, y, grad_outputs=torch.ones_like(u_y), create_graph=True)[0]
    u_t = torch.autograd.grad(u, t, grad_outputs=torch.ones_like(u), create_graph=True)[0]

    v_x = torch.autograd.grad(v, x, grad_outputs=torch.ones_like(v), create_graph=True)[0]
    v_xx = torch.autograd.grad(v_x, x, grad_outputs=torch.ones_like(v_x), create_graph=True)[0]
    v_y = torch.autograd.grad(v, y, grad_outputs=torch.ones_like(v), create_graph=True)[0]
    v_yy = torch.autograd.grad(v_y, y, grad_outputs=torch.ones_like(v_y), create_graph=True)[0]
    v_t = torch.autograd.grad(v, t, grad_outputs=torch.ones_like(v), create_graph=True)[0]

    p_x = torch.autograd.grad(p, x, grad_outputs=torch.ones_like(p), create_graph=True)[0]
    p_y = torch.autograd.grad(p, y, grad_outputs=torch.ones_like(p), create_graph=True)[0]

    f_u = u_t + model.lambda_1*(u * u_x + v * u_y) + p_x - model.lambda_2 * (u_xx + u_yy)
    f_v = v_t + model.lambda_1*(u * v_x + v * v_y) + p_y - model.lambda_2 * (v_xx + v_yy)

    #returning as column tensors
    return u.view(-1, 1), v.view(-1, 1), p.view(-1, 1), f_u.view(-1, 1), f_v.view(-1, 1)


def loss_complete(model, x, y, t, u_ori, v_ori):

    mse = nn.MSELoss()
    u_pred, v_pred, p_pred, f_u, f_v = compute_partials(model, x, y, t)
    aux = torch.zeros((x.shape[0], 1)).to(device)
    l1 = mse(u_ori, u_pred)
    l2 = mse(v_ori, v_pred)
    l3 = mse(f_u , aux)
    l4 = mse(f_v, aux)
    loss = (l1 + l2 + (l3 + l4))
    return loss


def tr_step(model, opt, x, y, t, u_ori, v_ori, lrlambdas = 1e-2):


    opt.zero_grad()

    l = loss_complete(model, x, y, t, u_ori, v_ori)
    l.backward()
    # model.lambda_1.data.sub_(lrlambdas * model.lambda_1.grad)
    # model.lambda_2.data.sub_(lrlambdas * model.lambda_2.grad)
    # model.lambda_1.grad.zero_()
    # model.lambda_2.grad.zero_()

    opt.step()

    return l.item()

def closure(model, opt, x, y, t, u_ori, v_ori, lrlambdas = 3e-3):
    """
    The closure function to use L-BFGS optimization method.
    """
    opt.zero_grad()
    # evaluating the MSE for the PDE
    loss = loss_complete(model, x, y, t, u_ori, v_ori)
    loss.backward()
    # model.lambda_1.data.sub_(lrlambdas * model.lambda_1.grad)
    # model.lambda_2.data.sub_(lrlambdas * model.lambda_2.grad)
    # model.lambda_1.grad.zero_()
    # model.lambda_2.grad.zero_()

    tr_loss.append(loss.item())
    global iter
    iter += 1
    if iter % 20 == 0:
        print(f"iteration: {iter}  loss: {loss.item():.6f}")
        print(f"lambda1 = {1.00}   || lambda_hat_1 = {model.lambda_1.item():.6f}")
        print(f"lambda2 = {0.01}   || lambda_hat_2 = {model.lambda_2.item():.6f}")

    if iter % 30==0:
        torch.save(model.state_dict(), f'models_trained/model_LBFGS_{iter}.pt')
    return loss

def tr_lbfgs(model, x, y, t, u_ori, v_ori):
    # Initialize the optimizer
    ls = [None, "strong_wolfe"]
    optimizer = torch.optim.LBFGS(model.parameters(),
                                  lr = 1,
                                  max_iter = 200000,
                                  max_eval = 50000,
                                  history_size = 50,
                                  tolerance_grad = 1e-05,
                                  tolerance_change = .5 * np.finfo(float).eps,
                                  line_search_fn = ls[1])

    # the optimizer.step requires the closure function to be a callable function without inputs
    # therefore we need to define a partial function and pass it to the optimizer
    closure_fn = partial(closure, model, optimizer, x, y, t, u_ori, v_ori)
    optimizer.step(closure_fn)


In [56]:
#L-BFGS training
tr_loss = []
tr_lbfgs(model_i, x_train, y_train, t_train, u_train, v_train)
    

iteration: 20  loss: 0.188374
lambda1 = 1.0   || lambda_hat_1 = 0.034982
lambda2 = 0.01   || lambda_hat_2 = -0.002702
iteration: 40  loss: 0.155113
lambda1 = 1.0   || lambda_hat_1 = 0.081731
lambda2 = 0.01   || lambda_hat_2 = -0.000476
iteration: 60  loss: 0.153212
lambda1 = 1.0   || lambda_hat_1 = 0.137316
lambda2 = 0.01   || lambda_hat_2 = 0.049825
iteration: 80  loss: 0.152903
lambda1 = 1.0   || lambda_hat_1 = 0.168323
lambda2 = 0.01   || lambda_hat_2 = 0.040999
iteration: 100  loss: 0.152742
lambda1 = 1.0   || lambda_hat_1 = 0.171194
lambda2 = 0.01   || lambda_hat_2 = 0.006046
iteration: 120  loss: 0.152628
lambda1 = 1.0   || lambda_hat_1 = 0.153889
lambda2 = 0.01   || lambda_hat_2 = -0.037654
iteration: 140  loss: 0.152512
lambda1 = 1.0   || lambda_hat_1 = 0.142928
lambda2 = 0.01   || lambda_hat_2 = -0.048972
iteration: 160  loss: 0.152445
lambda1 = 1.0   || lambda_hat_1 = 0.144420
lambda2 = 0.01   || lambda_hat_2 = -0.048392
iteration: 180  loss: 0.152398
lambda1 = 1.0   || lambd

In [ ]:
# training loop -- Adam
epochs = 10000
tr_loss = []
for epoch in tqdm(range(epochs)):
    l = tr_step(model_i, opt, x_train, y_train, t_train, u_train, v_train) 
    tr_loss.append(l)
    if epoch % 10 == 0:
        print(f"Training Loss at epoch {epoch + 1}: {tr_loss[-1]:.6f}")
    if epoch % 20 == 0:
        print(f"lambda1 = {1.}     || lambda_hat_1 = {model_i.lambda_1.item():.6f}")
        print(f"lambda2 = {0.01}   || lambda_hat_2 = {model_i.lambda_2.item():.6f}")

In [ ]:
opt = torch.optim.Adam(model_i.parameters(), lr = 1e-4)
# training loop -- Adam
epochs = 10000
tr_loss = []
for epoch in tqdm(range(epochs)):
    l = tr_step(model_i, opt, x_train, y_train, t_train, u_train, v_train) 
    tr_loss.append(l)
    if epoch % 10 == 0:
        print(f"Training Loss at epoch {epoch + 1}: {tr_loss[-1]:.6f}")
    if epoch % 20 == 0:
        print(f"lambda1 = {1.}     || lambda_hat_1 = {model_i.lambda_1.item():.6f}")
        print(f"lambda2 = {0.01}   || lambda_hat_2 = {model_i.lambda_2.item():.6f}")

In [20]:
model_i.lambda_1.item(), model_i.lambda_2.item()

(0.0009819908300414681, -1.0299073665009928e-06)

In [ ]:
t_unique = torch.unique(t_train)
t_unique

In [ ]:
# training in a seq2seq approach

for i in tqdm(range(t_unique.shape[0])):
    iter = 0
    tr_loss = []
    print(f"Training for snapshot t = {t_unique[i].item():.3f}")
    x_new, y_new, t_new, u_new, v_new = x_train[t_train == t_unique[i]], y_train[t_train == t_unique[i]],\
                                        t_train[t_train == t_unique[i]], u_train[t_train == t_unique[i]],\
                                        v_train[t_train == t_unique[i]]
    tr_lbfgs(model_i, x_new, y_new, t_new, u_new, v_new)
    print(f"Training snapshot t = {t_unique[i].item():.3f} finished.")
    plt.figure()
    plt.plot(tr_loss)
    plt.xlabel("#trainin-steps")
    plt.title(f"Traning Loss at snapshot t={t_unique[i].item():.3f}")
    plt.savefig(f"plots_training/loss_snap_t_{t_unique[i].item():.3f}.png")
    plt.close()


 

In [22]:
model_i.lambda_1, model_i.lambda_2

(Parameter containing:
 tensor([-0.0092], device='cuda:0', requires_grad=True),
 Parameter containing:
 tensor([0.0237], device='cuda:0', requires_grad=True))